# Idea
The idea is that we predict rotated linear dimension position from lines and text from the drawing.

# Thoughts
Basic logic is we split dataset to input lines or texts positions StartPoint, EndPoint, Position XYZ and predict dimension extension line poistion XLine1Point, XLine2Point XYZ.

We going to group samples by FileId. That is each sample will contain variable length data (attributes of variable number of  lines and text) and variable output data (variable number of dimensions).

I intend to [use RNN for it](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
import torch
from torch import nn
from torch.utils.data import Dataset, SubsetRandomSampler
from torch.nn import functional as F

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    
device

device(type='cuda')

# Dataset implementation

In [4]:
from dataset import DwgDataset
batch_size = 2

dwg_dataset = DwgDataset(pickle_file = 'test_dataset.pickle', batch_size = batch_size)

train_loader = dwg_dataset.train_loader
val_loader   = dwg_dataset.val_loader
test_loader  = dwg_dataset.test_loader

ent_features = dwg_dataset.entities.ent_features
dim_features = dwg_dataset.entities.dim_features

In [5]:
for c,d in dwg_dataset.entities:
    # print(c.shape, d.shape)
    # make sure we have every training sample not empty
    assert( c.shape[0] != 0)    

In [6]:
(a,b) = next(iter(train_loader))
print(len(a),len(b))
for (x,y) in iter(train_loader):
    for xx in x:
        print(xx.shape)
        pass
    for yy in y:
        print(yy.shape)
    # print(a.shape,b.shape)
    break

2 2
torch.Size([1670, 9])
torch.Size([157, 9])
torch.Size([83, 6])
torch.Size([93, 6])


# Model and training


In [7]:
from model import RnnDecoder, RnnEncoder
rnn_encoder = RnnEncoder(ent_features, 1024).to(device)
rnn_decoder = RnnDecoder(1024, dim_features).to(device)

In [8]:
(x,y) = next(iter(train_loader))
print('inputs','x:',len(x),'x',x[0].shape,'y:',len(y),'x',y[0].shape)
outs_numbers, learned = rnn_encoder(x)
print('learned shape:', learned.shape)
print("outs shape:", outs_numbers.shape, 'outs_value', outs_numbers)
# make sure something is passed
outs_numbers[0] = 42
decoded = rnn_decoder(outs_numbers, learned)
print('decoded:',len(decoded),'x', decoded[0].shape)

inputs x: 2 x torch.Size([8133, 9]) y: 2 x torch.Size([2702, 6])
learned shape: torch.Size([2, 1024])
outs shape: torch.Size([2, 1]) outs_value tensor([[4558],
        [ 453]], device='cuda:0', dtype=torch.int32)
decoded: 2 x torch.Size([42, 6])


In [9]:
%matplotlib inline
from train import train_model, plot_history

In [10]:
lr = 3e-4
epochs = 777
decoder_optimizer = torch.optim.Adam(rnn_decoder.parameters(), lr = lr)
encoder_optimizer = torch.optim.Adam(rnn_encoder.parameters(), lr = lr)
from chamfer_distance_loss import MyNumberLoss, MyChamferDistance
loss = MyNumberLoss()

loss_history, train_history, val_history = train_model(
    encoder = rnn_encoder, 
    decoder = rnn_decoder, 
    train_loader = train_loader,
    val_loader = val_loader,
    loss = loss,
    decoder_opt = decoder_optimizer,
    encoder_opt = encoder_optimizer,
    epochs = epochs)

plot_history(loss_history, train_history, val_history)

[0-0 @ 0.5 sec] Log10 Loss: 6.248590 Train err: 100.0%
[0-1 @ 0.6 sec] Log10 Loss: 3.367076 Train err: 100.0%
[0-2 @ 1.0 sec] Log10 Loss: 6.337623 Train err: 100.0%
[0-3 @ 2.7 sec] Log10 Loss: 7.971987 Train err: 100.0%
[0-4 @ 2.9 sec] Log10 Loss: 5.533806 Train err: 100.0%
[0-5 @ 2.9 sec] Log10 Loss: 4.020382 Train err: 100.0%
[0-6 @ 3.2 sec] Log10 Loss: 6.259095 Train err: 100.0%
[0-7 @ 3.5 sec] Log10 Loss: 6.179876 Train err: 100.0%
[0-8 @ 3.6 sec] Log10 Loss: 5.801537 Train err: 100.0%
[0-9 @ 4.0 sec] Log10 Loss: 6.921392 Train err: 100.0%
[0-10 @ 4.1 sec] Log10 Loss: 5.234001 Train err: 100.0%
Epoch validation accuracy: 0.000
[1-0 @ 6.3 sec] Log10 Loss: 7.977288 Train err: 100.0%
[1-1 @ 6.4 sec] Log10 Loss: 5.290841 Train err: 100.0%
[1-2 @ 6.5 sec] Log10 Loss: 4.432729 Train err: 100.0%
[1-3 @ 7.0 sec] Log10 Loss: 6.317747 Train err: 100.0%
[1-4 @ 7.3 sec] Log10 Loss: 6.150529 Train err: 100.0%
[1-5 @ 7.5 sec] Log10 Loss: 6.193013 Train err: 100.0%
[1-6 @ 7.7 sec] Log10 Loss: 5.8

RuntimeError: CUDA error: unspecified launch failure

In [11]:
from train import calculate_accuracy
i = 0
for (x, y) in iter(val_loader):
    outs, learned = rnn_encoder(x)
    decoded = rnn_decoder(outs, learned)
    
    yyy = []
    for yy in y:
        yyy.append(yy.shape[0])
    ppp = []
    for dd in decoded:
        ppp.append(dd.shape[0])
    
    print('yyy', yyy)
    print('prd', ppp)
    
    lv = loss(decoded, y)
    print ('loss', lv)

    acc = calculate_accuracy(decoded, y)
    print('acc', acc)

    i += 1
    print(i, '------------------------------')

RuntimeError: CUDA error: unspecified launch failure

In [ ]:
ii = pd.DataFrame(x[0].cpu().detach().numpy())
print(ii.head())
yy = pd.DataFrame(y[0].cpu().detach().numpy())
print(len(yy))
print(yy.head())
pp = pd.DataFrame(prediction[0].cpu().detach().numpy())
print(len(pp))
print(pp.head())

In [ ]:
from train import calculate_accuracy
torch.cuda.empty_cache()

rnn_encoder.eval()
rnn_decoder.eval()

test_accuracies = []
for (x,y) in test_loader:
    with torch.no_grad():
        out, hidden = rnn_encoder(x)
        prediction = rnn_decoder(out, hidden)
        accuracy = calculate_accuracy(prediction, y)
        test_accuracies.append(accuracy)
        
mean_test_accuracy = np.mean(test_accuracies)
print('Accuracy on testing: {0:2.3f}'.format(mean_test_accuracy))